# Your first scraper

- Repository: `Wikipedia Scrapper`
- Type of Challenge: `Learning`
- Duration: `3 days`
- Submission: `23/06/2023 4:30 PM`

- Team challenge : `Solo`

## The Mission

In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

## Learning objectives

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

* Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.
* Activate it.
* You will find more info about virtual environments in the course content and on the web.
* Note: BEFORE pushing changes into your repo explore what the .gitignore file is and add your environment to it. 

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [325]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url="https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url="https://country-leaders.onrender.com/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req=requests.get(status_url)
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code==200:
    print(req.text)
else:
    print(req.status_code)

"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted. The nice thing for us is that JSON files follow a format we already know in Python...dictionaries!

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [326]:
# Set the countries_url variable (1 line)
countries_url="https://country-leaders.onrender.com/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req=requests.get(countries_url)
# Get the JSON content and store it in the countries variable (1 line)
countries=req.json()
# display the request's status code and the countries variable (1 line)
print(req.status_code,countries)

403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [327]:
# Set the cookie_url variable (1 line)
cookie_url="https://country-leaders.onrender.com/cookie"

# Query the enpoint, set the cookies variable and display it (2 lines)
response=requests.get(cookie_url)
cookie_value=response.cookies.get('user_cookie')
out_put_cookie={"user_cookie":cookie_value}
print(out_put_cookie)

{'user_cookie': 'c2459029-9bfc-4b31-85de-7bf750a332a5'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [328]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
headers=out_put_cookie
# display the countries variable (1 line)
countries=requests.get(countries_url,cookies=headers)
countries.json()


['us', 'be', 'fr', 'ma', 'ru']

Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [329]:
# Set the leaders_url variable (1 line)
leader_url="https://country-leaders.onrender.com/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders=requests.get(leader_url,cookies=headers)
# display the leaders variable (1 line)
leaders.json()

{'message': 'Please specify a country'}

It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [330]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
param={"country":"us"}
leaders=requests.get(leader_url,cookies=headers,params=param)
# display the leaders variable (1 line)
print(leaders.json())


[{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'place_of_bi

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [331]:
# 4 lines
leaders_per_country=dict()
for country in countries.json():
    leaders=requests.get(leader_url,cookies=headers,params={"country":country})
    leaders_per_country[country]=leaders.json()
print(leaders_per_country)


{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

In [332]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [333]:
# < 15 lines
def get_leaders():
    #define the urls
    leader_url="https://country-leaders.onrender.com/leaders"
    #get the cookies
    cookie_url="https://country-leaders.onrender.com/cookie"
    response=requests.get(cookie_url)
    cookie_value=response.cookies.get('user_cookie')
    print(cookie_value)
    #get the countries
    headers={"user_cookie":cookie_value}
    countries=requests.get(countries_url,cookies=headers)
    countries.json()
    #loop over the country and save it to the dictionary
    leaders_per_country=dict()
    for country in countries.json():
        leaders=requests.get(leader_url,cookies=headers,params={"country":country})
        leaders_per_country[country]=leaders.json()
    

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [334]:
# 2 lines
leaders_per_country[country]=leaders.json()
print(leaders_per_country)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [335]:
# 3 lines
leader1_url='https://en.wikipedia.org/wiki/James_Madison'
req_leader1=requests.get(leader1_url)
print(req_leader1.content)

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>James Madison - Wikipedia</title>\n<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";(function(){var cookie=document.cookie.match(/(?:^|; )enwikimwclie

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [336]:
# 3 lines
from bs4 import BeautifulSoup
soup=BeautifulSoup(req_leader1.content,'html')
soup.get_text()
soup.prettify()




'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   James Madison - Wikipedia\n  </title>\n  <script>\n   document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";(function(){var cookie=document.cookie.match(

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [337]:
# 2 lines
for paragraph in soup.find_all('p'):
    print(paragraph.text)
    





James Madison Jr. (March 16, 1751[b] – June 28, 1836) was an American statesman, diplomat, and Founding Father who served as the fourth president of the United States from 1809 to 1817. Madison is hailed as the "Father of the Constitution" for his pivotal role in drafting and promoting the Constitution of the United States and the Bill of Rights.

Madison was born into a prominent slave-owning planter family in Virginia. He served as a member of the Virginia House of Delegates and the Continental Congress during and after the American Revolutionary War. Dissatisfied with the weak national government established by the Articles of Confederation, he helped organize the Constitutional Convention, which produced a new constitution designed to strengthen republican government against democratic assembly. Madison's Virginia Plan was the basis for the convention's deliberations, and he was an influential voice at the convention. He became one of the leaders in the movement to ratify the C

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [338]:
for paragraph in soup.find_all('p'):
        if paragraph.find('b'):
                first_paragraph=paragraph.text
                print(first_paragraph)
                break
    
    

James Madison Jr. (March 16, 1751[b] – June 28, 1836) was an American statesman, diplomat, and Founding Father who served as the fourth president of the United States from 1809 to 1817. Madison is hailed as the "Father of the Constitution" for his pivotal role in drafting and promoting the Constitution of the United States and the Bill of Rights.



In [339]:
# 10 lines
from bs4 import BeautifulSoup
wikipedia_url='https://en.wikipedia.org/wiki/James_Madison'
def get_first_paragraph(wikipedia_url):    
    req_leader1=requests.get(wikipedia_url)
    soup=BeautifulSoup(req_leader1.content,'html')
    for paragraph in soup.find_all('p'):
        if paragraph.find('b'):
                first_paragraph=paragraph.text
                print(first_paragraph) 
                break
                
get_first_paragraph(wikipedia_url)
#  print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

James Madison Jr. (March 16, 1751[b] – June 28, 1836) was an American statesman, diplomat, and Founding Father who served as the fourth president of the United States from 1809 to 1817. Madison is hailed as the "Father of the Constitution" for his pivotal role in drafting and promoting the Constitution of the United States and the Bill of Rights.



In [340]:
# Test: 3 lines


### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [358]:
# 3 lines
import re
reg=r"[^\[b\]]"
clean_fir_para=re.findall(reg,first_paragraph)
print(clean_fir_para)



['J', 'a', 'm', 'e', 's', ' ', 'M', 'a', 'd', 'i', 's', 'o', 'n', ' ', 'J', 'r', '.', ' ', '(', 'M', 'a', 'r', 'c', 'h', ' ', '1', '6', ',', ' ', '1', '7', '5', '1', '\xa0', '–', ' ', 'J', 'u', 'n', 'e', ' ', '2', '8', ',', ' ', '1', '8', '3', '6', ')', ' ', 'w', 'a', 's', ' ', 'a', 'n', ' ', 'A', 'm', 'e', 'r', 'i', 'c', 'a', 'n', ' ', 's', 't', 'a', 't', 'e', 's', 'm', 'a', 'n', ',', ' ', 'd', 'i', 'p', 'l', 'o', 'm', 'a', 't', ',', ' ', 'a', 'n', 'd', ' ', 'F', 'o', 'u', 'n', 'd', 'i', 'n', 'g', ' ', 'F', 'a', 't', 'h', 'e', 'r', ' ', 'w', 'h', 'o', ' ', 's', 'e', 'r', 'v', 'e', 'd', ' ', 'a', 's', ' ', 't', 'h', 'e', ' ', 'f', 'o', 'u', 'r', 't', 'h', ' ', 'p', 'r', 'e', 's', 'i', 'd', 'e', 'n', 't', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'U', 'n', 'i', 't', 'e', 'd', ' ', 'S', 't', 'a', 't', 'e', 's', ' ', 'f', 'r', 'o', 'm', ' ', '1', '8', '0', '9', ' ', 't', 'o', ' ', '1', '8', '1', '7', '.', ' ', 'M', 'a', 'd', 'i', 's', 'o', 'n', ' ', 'i', 's', ' ', 'h', 'a', 'i', 'l', 'e', '

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [342]:
# 10 lines


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [343]:
# < 20 lines


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [344]:
# < 20 lines

In [345]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [346]:
# < 25 lines



Check the output of your function again.

In [347]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [348]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [349]:
# <25 lines


Test your new functions.



## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [350]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [351]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [352]:
# 3 lines


In [353]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

### Constraints

#### Code style

- Your code should be **commented**. You can use a docstring to explain what your functions are doing. E.g. 

```python
def add(number_one, number_two):
    """
    Function that will perform the add operation between two numbers (in params).
    """
    result = number_one + number_two
    return result
```


- Your code should be **cleaned of any commented, unused code**.

### GitHub Repository

- Push your code on a GitHub repository. The name is at the top of this page.
- Your repository should only contain the files specified.
- Your `README.md` should be nice and complete.
  It should include the following outline:
  - Description (What is the project about? What is the goal? What is your approach?)
  - Installation (What are the requirements to use your code?)
  - Usage (How does someone use your code?)
  - Timeline (How long did the project take?)
  - Personal situation (e.g. This project was done as part of the AI Boocamp at BeCode.org.)

Feel free to add more information.

## Evaluation criteria

| Criteria       | Indicator                                                   | Yes/No |
| -------------- | ----------------------------------------------------------- | ------ |
| 1. Is complete | The student has realized all must-have features.            |        |
|                | There is a published GitHub repo available.                 |        |
|                | The scrapper starts by running `python main.py` in the terminal |        |
| 2. Is correct  | The code is well typed.                                     |        |
|                | All the constraints are respected                           |        |


## To go further (if time permits)

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!

![wikepedia_gif](https://media.giphy.com/media/mADzY6J0QjniQl9gs5/giphy.gif)